In [4]:
import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkstudy.EventProfiler as ep

import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import copy

#第一周作业
def simulate(raw_data, ls_symbols , alloct):
    daily_data = raw_data.copy();
    daily_data[:]=1
    daily_data['polio']=1;
    daily_data['dailyRet']=0;
    for i in range(1,len(raw_data),1):
        daily_data.iloc[i] = (raw_data.iloc[i])/raw_data.iloc[0] 
        fixedDay = daily_data.iloc[i][ls_symbols[0]] * alloct[0] + daily_data.iloc[i][ls_symbols[1]] * alloct[1] +daily_data.iloc[i][ls_symbols[2]] * alloct[2]+daily_data.iloc[i][ls_symbols[3]] * alloct[3]
        daily_data.iloc[i] = daily_data.iloc[i].set_value('polio' , fixedDay)
        daily_data.iloc[i] = daily_data.iloc[i].set_value('dailyRet' , (daily_data.iloc[i]['polio'] - daily_data.iloc[i-1]['polio']) /daily_data.iloc[i-1]['polio'])
    return math.sqrt(252) * daily_data['dailyRet'].mean() / daily_data['dailyRet'].std()

    
def bestPolio():
    ls_symbols = ['BRCM', 'ADBE', 'AMD', 'ADI']
    dt_start = dt.datetime(2010, 1, 1)
    dt_end = dt.datetime(2010, 12, 31) 
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    raw_data = d_data['close']
    res = [];
    for i in range(0,10,1):
        for j in range(0,10,1):
            for k in range(0,10-i-j,1):
                l = 1 - round(i,1)/10 - round(j,1)/10 - round(k,1)/10
                if l >= 0:
                    res.append(simulate(raw_data, ls_symbols , [round(i,1)/10, round(j,1)/10, round(k,1)/10, l]))
    npres=np.array(res)
    print npres.max()
    
#第一周作业调用函数    
#bestPolio()

#第二周作业
def preEventData():
    dataobj = da.DataAccess('Yahoo')
    ls_symbols = dataobj.get_symbols_from_list("sp5002012")
    ls_symbols.append('SPY')
    dt_start = dt.datetime(2008, 1, 1)
    dt_end = dt.datetime(2009, 12, 31) 
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    for s_key in ls_keys:
        d_data[s_key] = d_data[s_key].fillna(method = 'ffill')
        d_data[s_key] = d_data[s_key].fillna(method = 'bfill')
        d_data[s_key] = d_data[s_key].fillna(1.0)
    return d_data,ls_symbols

def eventStudy():
    d_data,ls_symbols = preEventData()#pd.read_csv('sp5002008.csv',parse_dates=True)
  
    print 'create event matrix'
    df_close = d_data['close']
    ts_market = df_close['SPY']
    # Creating an empty dataframe
    df_events = copy.deepcopy(df_close)
    df_events = df_events * np.NAN
    
    
    ldt_timestamps = df_close.index 
    
    print 'compute event'
    
    for s_sym in ls_symbols:
        for i in range(1,len(ldt_timestamps)):
            f_symprice_today = df_close[s_sym].ix[ldt_timestamps[i]]
            f_symprice_yest = df_close[s_sym].ix[ldt_timestamps[i - 1]]
            f_marketprice_today = ts_market.ix[ldt_timestamps[i]]
            f_marketprice_yest = ts_market.ix[ldt_timestamps[i - 1]]
            f_symreturn_today = (f_symprice_today / f_symprice_yest) - 1
            f_marketreturn_today = (f_marketprice_today / f_marketprice_yest) - 1
            if f_symreturn_today <= -0.03 and f_marketreturn_today >= 0.02:
                df_events[s_sym].ix[ldt_timestamps[i]] = 1
    print 'Creating Study'
    ep.eventprofiler(df_events, d_data, i_lookback=20, i_lookforward=20,s_filename='MyEventStudy2012.pdf',b_market_neutral=True, b_errorbars=True,s_market_sym='SPY')
    
eventStudy()
   
  
    
   


create event matrix
compute event
Creating Study
